In [5]:
from moviepy import VideoFileClip
import os

In [9]:
video = "../data/video_data/videoplayback.mp4"
video_name = os.path.basename(video)

In [10]:
video = VideoFileClip(video)
audio = video.audio

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [548, 640], 'bitrate': 604, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(Constrained Baseline)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 192, 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 6.12, 'bitrate': 794, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Constrained Baseline)', 'video_size': [548, 640], 'video_bitrate': 604, 'video_fps': 

In [11]:
audio.write_audiofile(f"../data/converted_audio_from_video/{video_name[:-4]}.mp3")

MoviePy - Writing audio in ../data/converted_audio_from_video/videoplayback.mp3


MoviePy - Done.


# Transcription Part (From test_transcription.ipynb)

In [12]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

In [ ]:
model_path = "../models/mms"
processor = Wav2Vec2Processor.from_pretrained(model_path)
model = Wav2Vec2ForCTC.from_pretrained(model_path)

In [ ]:
def load_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    return waveform, sample_rate

In [ ]:
def predict(audio_path):
    waveform, sample_rate = load_audio(audio_path)

    # Resample if necessary
    target_sample_rate = 16000
    if sample_rate != target_sample_rate:
        transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        waveform = transform(waveform)

    # Convert to tensor
    input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", sampling_rate=target_sample_rate).input_values

    # Run model
    with torch.no_grad():
        logits = model(input_values).logits

    # Decode
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    
    return transcription

In [ ]:
audio_file = (f"../data/converted_audio_from_video/{video_name[:-4]}.mp3")
transcription = predict(audio_file)
print("Transcription:", transcription)